In [21]:
import pandas as pd
import water.utils.parse as p
import os
import numpy as np

In [23]:
folder = './data/input/reserva'
folder_new = './data/input/cleaned'
folder_input = './data/input/cleaned'
folder_output = './data/output'

In [3]:
for root, dirs, files in os.walk(folder):
    for file in files:
        if file.endswith(".pdf"):
            # Copy the file to the folder_new
            filename_new = folder_new + '/' + file
            # Get the old file, expanding the sub-path
            filename_old = os.path.join(root, file)
            # Copy from old to new if new does not exist
            if not os.path.exists(filename_new):
                print(filename_old)
                os.system('cp ' + filename_old + ' ' + filename_new)   

./data/input/reserva/2024_02_08.pdf
./data/input/reserva/2024_01_29.pdf
./data/input/reserva/2024_01_28.pdf
./data/input/reserva/2024_01_31.pdf
./data/input/reserva/2024_01_30.pdf
./data/input/reserva/2024_01_27.pdf
./data/input/reserva/2024_02_03.pdf
./data/input/reserva/2024_02_02.pdf
./data/input/reserva/2024_02_01.pdf
./data/input/reserva/2024_02_05.pdf
./data/input/reserva/2024_02_04.pdf
./data/input/reserva/2024_02_06.pdf
./data/input/reserva/2024_02_07.pdf


In [7]:
import importlib
importlib.reload(p)

filenames_raw = sorted(os.listdir(folder_input))
# filenames = [f for f in filenames_raw if p.filter_weekly(f)]
filenames = filenames_raw[-400:][::-1]

for i, filename in enumerate(filenames):
    if i % 50 == 0:
        print(f'{i} of {len(filenames)}: {filename} / share completed: {round(i/len(filenames)*100, 2)}%')
    full_filename = os.path.join(folder_new, filename)
    filename_output = filename.replace('pdf', 'csv')
    filename_output = os.path.join(folder_output, filename_output)
    if not os.path.exists(filename_output):
        try:
            tables = p.get_tables(full_filename)
            df_raw = p.get_df(tables)
            df = p.clean_df(df_raw)
            df.to_csv(filename_output, index=False)
        except Exception as e:
            print(full_filename)
            print(e)
            pass

0 of 400: 2024_02_08.pdf / share completed: 0.0%
50 of 400: 2023_12_19.pdf / share completed: 12.5%
100 of 400: 2023_10_30.pdf / share completed: 25.0%
150 of 400: 2023_09_10.pdf / share completed: 37.5%
200 of 400: 2023_07_20.pdf / share completed: 50.0%
250 of 400: 2023_05_31.pdf / share completed: 62.5%
300 of 400: 2023_04_10.pdf / share completed: 75.0%
350 of 400: 2023_02_19.pdf / share completed: 87.5%


In [24]:
import os

folder = './data/datasets'
!ls -l $folder

total 29176
-rw-r--r--@ 1 cg  staff  4187112 Mar 18 09:29 all_parsed.csv
-rw-r--r--@ 1 cg  staff  3829541 Mar 18 09:26 all_parsed_cleaned.csv
-rw-r--r--@ 1 cg  staff   912994 Feb 11 14:15 monthly.csv
-rw-r--r--@ 1 cg  staff   623424 Feb 11 14:15 monthly_cleaned.csv
-rw-r--r--@ 1 cg  staff  4186790 Feb 11 14:09 monthly_corrected.csv
-rw-r--r--@ 1 cg  staff   401607 Feb 11 14:15 recent.csv
-rw-r--r--@ 1 cg  staff   397212 Feb 11 14:15 recent_cleaned.csv


In [26]:
filenames_all = sorted(os.listdir(folder_output))
df_all = p.get_full_df(filenames_all)
df_all.to_csv('./data/datasets/all_parsed.csv', index=False)
df_all = p.correct_issues(df_all).sort_values(['ds', 'province', 'reservoir'])

def remove_bad_rows(df):
    df_all = p.add_cols(df)

    df_all['date_lag'] = df_all.groupby(['province', 'reservoir'])['date'].shift(1)

    cols = ['rainfallsince', 'stored_hm3', 'capacity_hm3']
    for var in ['rainfallsince', 'stored_hm3']:
        df_all[f'{var}_diff'] = df_all.groupby(['province', 'reservoir'])[var].diff()
        df_all[f'{var}_diff_0'] = df_all[f'{var}_diff']
        for lags in range(1, 10):
            df_all[f'{var}_diff_{lags}'] = df_all.groupby(['province', 'reservoir'])[f'{var}_diff'].shift(lags)
            
    df_all['bad_data'] = df_all.rainfallsince_diff_0 < -10
    df_all['bad_data_for_year'] = df_all.groupby(['province', 'reservoir', 'year_climatic'])['bad_data'].transform('any')
    df_all['problem_month'] = df_all.month.isin([1, 10, 11, 12])
    df_all['bad_data_and_month'] = df_all.bad_data & df_all.problem_month

    df_all['stored_hm3_diff_relative'] = df_all.stored_hm3_diff / df_all.capacity_hm3
    df_reg = df_all.query('~bad_data_and_month').query('rainfallsince_diff >=0').copy()

    df_reg['bad_data_for_year'] = df_reg.groupby(['province', 'reservoir', 'year_climatic'])['bad_data'].transform('any')
    df_reg = df_reg.sort_values(['province', 'reservoir', 'date'])
    df_reg['date_diff'] = (df_reg.date - df_reg.date_lag).dt.days
    df_reg[['province', 'reservoir', 'date']].head(10)

    df_reg['bad_data_for_year'] = df_reg.groupby(['province', 'reservoir', 'year_climatic'])['bad_data'].transform('any')
    assert df_reg.bad_data_for_year.sum() == 0

    def add_lags(df, var_name, lags=np.arange(-5, 5), groups=['province', 'reservoir']):
        for lag in lags:
            df[f'{var_name}_lag_{lag}'] = df.groupby(groups)[var_name].shift(lag)
            
        lag_vars = [f'{var_name}_lag_{lag}' for lag in lags]
        return df, lag_vars

    df_reg['suspicious_storage'] = (np.abs(df_reg['stored_hm3_diff']) > 2) & (np.abs(df_reg['stored_hm3_diff_relative']) > 0.05)
    df_reg['high_rain'] = (df_reg.rainfallsince_diff > 10) | (df_reg.rainfallsince_diff_1 > 10)
    df_reg['bad_storage'] = df_reg.suspicious_storage & (~df_reg.high_rain)
    df_reg, lag_vars = add_lags(df_reg, 'bad_storage')

    df_reg['surrounding_bad_storage'] = df_reg[lag_vars].max(axis=1)
    df_reg = df_reg.query('surrounding_bad_storage==0').copy()
    return df_reg

df_removed = remove_bad_rows(df_all).sort_values(['ds', 'province', 'reservoir'])

df_removed = df_removed[df_all.columns].sort_values(['reservoir', 'ds'])

len(df_removed), len(df_all)
df_removed.to_csv(f'{folder}/all_parsed_cleaned.csv', index=False)
len(df_removed)

42036

In [32]:
def pick_monthly(df):
    df_monthly = df[df.ds.str.slice(8, 10) == "01"].copy()
    return df_monthly

# Pick only the first day of the month from df_all
df_monthly = pick_monthly(df_all)
df_monthly_cleaned = pick_monthly(df_removed)

df_monthly.to_csv('./data/datasets/monthly.csv', index=False)
df_monthly_cleaned.to_csv('./data/datasets/monthly_cleaned.csv', index=False)


num_tail = 60
ds_recent = df_all.ds.unique()[-num_tail:]
df_recent = df_all[df_all.ds.isin(ds_recent)].copy()
df_recent_cleaned = df_removed[df_removed.ds.isin(ds_recent)].copy()

df_recent.to_csv('./data/datasets/recent.csv', index=False)
df_recent_cleaned.to_csv('./data/datasets/recent_cleaned.csv', index=False)

In [24]:
filename_all = f'{folder}/all_parsed_cleaned.csv'

df_all = pd.read_csv(filename_all)

FileNotFoundError: [Errno 2] No such file or directory: './data/input/reserva/all_parsed_cleaned.csv'

In [31]:
df_all = pd.read_csv('../../../../data/datasets/all_parsed_cleaned.csv')

In [34]:
# Store df_all in s3

BUCKET = 'andalucianwater'

# Store the df in s3
import boto3
import io

s3 = boto3.client('s3')
filename = '../../../../data/datasets/all_parsed_cleaned.csv'

with open(filename, 'rb') as f:
    s3.upload_fileobj(f, BUCKET, 'datasets/all_parsed_cleaned.csv')
    


In [33]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 41.6 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 6.8 MB/s eta 0:00:00
